In [13]:
import os
from datetime import datetime
import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils

# ========== Paths ==========
data_root = "./dataset/raw/img_align_celeba/img_align_celeba"
save_dir = "./dataset/save/"
os.makedirs(save_dir, exist_ok=True)

# ========== Hyperparams ==========
batch_size = 512
lr = 2e-4
num_epochs = 100
latent_dim = 128
sample_every = 5
num_sample_images = 8
image_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
])

In [14]:
# import os

# # Install the Kaggle API client
# !pip install kaggle

# # Create the .kaggle directory and copy the kaggle.json file
# !mkdir -p ~/.kaggle
# # You will need to upload your kaggle.json file to your Colab environment.
# # In the left sidebar, click on the folder icon, then click on the upload icon.
# # Upload your kaggle.json file there.
# # Once uploaded, move it to the correct directory:
# # !mv kaggle.json ~/.kaggle/

# # If you have stored your Kaggle API key and username as Colab secrets, you can use the following:
# from google.colab import userdata

# os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
# os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# # Set permissions for the kaggle.json file
# !chmod 600 ~/.kaggle/kaggle.json

# # Download the dataset
# !kaggle datasets download -d jessicali9530/celeba-dataset -p ./dataset/raw --unzip

In [15]:
# ========== Dataset ==========
class CelebADataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.paths = sorted(glob.glob(os.path.join(root, "*.jpg")))
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

dataset = CelebADataset(root=data_root, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True,
                    num_workers=2, pin_memory=True)
print("Num Images: ", len(dataset))

# ========== Utilities ==========
def save_checkpoint(model, optim, epoch, path):
    state = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optim_state": optim.state_dict()
    }
    torch.save(state, path)


def save_image_grid(tensor, filename, nrow=8):
    tensor = torch.clamp(tensor, 0, 1)
    utils.save_image(tensor, filename, nrow=nrow, padding=2)

Num Images:  202599


In [16]:
# ========== Loss ==========
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum') / x.size(0)
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)

    # β-VAE: 可以引入一个权重来平衡两项，初期可以设 beta=1
    beta = 1.0

    return recon_loss + beta * kld, recon_loss, kld


In [17]:
# ========== VAE Model ==========
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.ReLU(),
            nn.Flatten(),
        )
        self.fc_mu = nn.Linear(256 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(256 * 4 * 4, latent_dim)

        self.decoder_input = nn.Linear(latent_dim, 256 * 4 * 4)
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1),
            nn.Sigmoid(),
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_input(z)
        return self.decoder(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [18]:
from tqdm import tqdm
model = VAE(latent_dim=latent_dim).to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)
global_step = 0
for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    epoch_recon = 0.0
    epoch_kld = 0.0

    for batch_idx, imgs in enumerate(tqdm(loader)):
        imgs = imgs.to(device, non_blocking=True)
        optim.zero_grad()
        recon_imgs, mu, logvar = model(imgs)
        loss, recon_l, kld = vae_loss(recon_imgs, imgs, mu, logvar)
        loss.backward()
        optim.step()

        epoch_loss += loss.item()
        epoch_recon += recon_l.item()
        epoch_kld += kld.item()
        global_step += 1

        if batch_idx % 100 == 0:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] "
                    f"Epoch {epoch}/{num_epochs} Batch {batch_idx}/{len(loader)} "
                    f"Loss {loss.item():.4f} "
                    f"(recon {recon_l.item():.4f}, kld {kld.item():.4f})")

    n_samples = len(loader.dataset)
    print(f"=== Epoch {epoch} finished. Avg loss: {epoch_loss / n_samples:.4f} "
            f"(recon {epoch_recon / n_samples:.4f}, kld {epoch_kld / n_samples:.4f}) ===")

    # 保存样本
    if epoch % sample_every == 0 or epoch == 1:
        # 保存检查点
        ckpt_path = os.path.join(save_dir, f"vae_epoch{epoch}.pth")
        save_checkpoint(model, optim, epoch, ckpt_path)
        model.eval()
        with torch.no_grad():
            # 重建样本
            imgs = next(iter(loader))
            imgs = imgs.to(device)[:num_sample_images]
            recon_imgs, _, _ = model(imgs)
            combined = torch.cat([imgs, recon_imgs], dim=0)  # 不再需要clamp
            save_image_grid(combined, os.path.join(save_dir, f"recon_epoch{epoch}.png"), nrow=8)

            # 生成样本
            z = torch.randn(num_sample_images, latent_dim).to(device)
            samples = model.decode(z)
            save_image_grid(samples, os.path.join(save_dir, f"sample_epoch{epoch}.png"), nrow=8)
        model.train()

print("Training complete.")

  0%|          | 1/396 [00:01<09:41,  1.47s/it]

[2025-10-06 09:07:41] Epoch 1/100 Batch 0/396 Loss 1135.2826 (recon 1135.2656, kld 0.0170)


 25%|██▌       | 100/396 [00:48<02:05,  2.36it/s]

[2025-10-06 09:08:28] Epoch 1/100 Batch 100/396 Loss 640.2681 (recon 612.3834, kld 27.8847)


 51%|█████     | 200/396 [01:36<02:06,  1.55it/s]

...

[2025-10-06 09:28:11] Epoch 7/100 Batch 200/396 Loss 200.4318 (recon 144.4507, kld 55.9811)


 76%|███████▌  | 301/396 [02:21<00:40,  2.33it/s]

[2025-10-06 09:28:57] Epoch 7/100 Batch 300/396 Loss 200.3324 (recon 143.6122, kld 56.7202)


 94%|█████████▍| 373/396 [02:54<00:10,  2.14it/s]


KeyboardInterrupt: 